In [1]:
import pandas as pd
import importlib
from pathlib import Path
import transformers
from transformers import AutoTokenizer, AutoModel
import torch
import pytorch_lightning as pl

In [2]:
import dsci_2022

In [3]:
!pwd

/lustre06/project/6001735/dsci_2022/notebooks


In [4]:
transformers.logging.set_verbosity_error()
TRANSFORMER_CACHE = Path("../models/transformer_cache")
data = pd.read_pickle('../data/labels.pkl')

In [5]:
trainData = data[data.paper_id<=1150]
testData = data[data.paper_id>1640]

In [6]:
pretrained_model = AutoModel.from_pretrained(
    '../resources/checkpoints/bert-base-uncased.pt'
)
tokenizer = AutoTokenizer.from_pretrained(
    '../resources/checkpoints/bert-base-uncased-tokenizer.pt'
)

In [7]:
%%time
importlib.reload(dsci_2022.chunksumm)

data_module = dsci_2022.chunksumm.SummDataModule(
    trainData = trainData[trainData.paper_id<=1000], 
    valData = trainData[trainData.paper_id>1000],
    testData = testData,
    tokenizer = tokenizer,
    SuMMDataset = dsci_2022.chunksumm.SuMM_with_tokenizer,
    process_paper_level = True
)

data_module.setup()
print(
    f"train_len : {data_module.train_len} , test_len : {data_module.test_len}, val_len : {data_module.val_len} "
)
print(next(iter(data_module.train_dataloader()))["input_ids"].shape)
print(next(iter(data_module.val_dataloader()))["input_ids"].shape)
print(next(iter(data_module.test_dataloader()))["input_ids"].shape)

train_len : 225584 , test_len : 2548, val_len : 33702 
torch.Size([8, 9359])
torch.Size([8, 20024])
torch.Size([8, 8878])
CPU times: user 38.2 ms, sys: 96.7 ms, total: 135 ms
Wall time: 3.7 s


In [8]:
# This is required to initialize the backend-model (bert) which is a pretrained model.
importlib.reload(dsci_2022.chunksumm)
pretrained_model = AutoModel.from_pretrained('../resources/checkpoints/bert-base-uncased.pt')
tokenizer = AutoTokenizer.from_pretrained('../resources/checkpoints/bert-base-uncased-tokenizer.pt')
model = dsci_2022.chunksumm.CHUNKSUMM_CONV(
    model = pretrained_model,
    learning_rate = 1e-6,
    enable_chunk = True
)

/home/jarobyte/envs/detests/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/jarobyte/envs/detests/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:261: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(


In [9]:


# pl.Trainer(gpus=-1,logger=None).test(model=model,
#                                             dataloaders=data_module.test_dataloader())

In [10]:
batch = next(iter(data_module.train_dataloader()))

In [11]:
# %%time
# model.cpu()
# out = model(
#     batch["input_ids"].cpu(), 
#     batch["attention_mask"].cpu(), 
#     batch["token_type_ids"].cpu()
# )
# out.shape

In [12]:
%%time
model.cuda()
out = model(
    batch["input_ids"].cuda(), 
    batch["attention_mask"].cuda(), 
    batch["token_type_ids"].cuda()
)
out.shape

CPU times: user 1.65 s, sys: 1.31 s, total: 2.95 s
Wall time: 3.37 s


torch.Size([8, 2, 9359])

In [13]:
#torch.where(batch['targets'].bool(),torch.tensor([1,0]),torch.tensor([0,1]))

In [14]:
new_targets = model.expand_targets(batch['targets'])

In [15]:
batch['targets'][-2,0]

tensor(1.)

In [16]:
new_targets

tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]], device='cuda:0')

In [17]:
# new_targets.reshape_as(out)

In [18]:
new_targets

tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]], device='cuda:0')

In [19]:
!nvidia-smi

Sat Jul  9 15:45:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:41:00.0 Off |                    0 |
| N/A   29C    P0    58W / 400W |   5036MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------